# Exercises for Apache Spark™ and Scala Workshops PART - III

This are my own solutions version in PySpark of the Exercises proposed by Jacek Laskowski in https://github.com/jaceklaskowski/spark-workshop/tree/gh-pages/exercises

## Exercises:

21. [Using pivot to generate a single-row matrix](#21) 
22. [Using pivot for Cost Average and Collecting Values](#22)
23. [Pivoting on Multiple Columns](#23)
24. [Generating Exam Assessment Report](#24) 
25. [Flattening Dataset from Long to Wide Format](#25)
26. [Finding 1st and 2nd Bestsellers Per Genre](#26)
27. [Calculating aggregations](#27) 
28. [Calculating Running Total / Cumulative Sum](#28)
29. [Calculating Difference Between Consecutive Rows Per Window](#29)
30. [Converting Arrays of Strings to String](#30)

# SET UP

In [1]:
!pip install findspark

import findspark
findspark.init()

In [2]:
# Cargar Pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf


spark = SparkSession.builder.appName("Test_spark").master("local[*]").getOrCreate()

spark

# LIBRARIES

In [3]:
from pyspark.sql.functions import *
from pyspark.sql import Window
from pyspark.sql.types import *

## 21. Using pivot to generate a single-row matrix <a id='21'></a>

In [8]:
df_input = spark.createDataFrame([
[20090622,458],
[20090624,31068],
[20090626,151],
[20090629,148],
[20090914,453]],("update","cc"))

df_input.show()

+--------+-----+
|  update|   cc|
+--------+-----+
|20090622|  458|
|20090624|31068|
|20090626|  151|
|20090629|  148|
|20090914|  453|
+--------+-----+



In [16]:
# Set a constant to groupby
df_output = df_input.withColumn("constant",lit(1))
# Pivot the table to transpose it
df_output = (df_output
             .groupby("constant")
             .pivot("update")
             .agg(min("cc")))
# Renamed
df_output = (df_output
             .withColumn("constant",lit("cc"))
             .withColumnRenamed("constant","update"))
df_output.show()

+------+--------+--------+--------+--------+--------+
|update|20090622|20090624|20090626|20090629|20090914|
+------+--------+--------+--------+--------+--------+
|    cc|     458|   31068|     151|     148|     453|
+------+--------+--------+--------+--------+--------+



## 22. Using pivot for Cost Average and Collecting Values <a id='22'></a>

In [18]:
df_input = spark.createDataFrame([
  [0, "A", 223, "201603", "PORT"],
  [0, "A", 22, "201602", "PORT"],
  [0, "A", 422, "201601", "DOCK"],
  [1, "B", 3213, "201602", "DOCK"],
  [1, "B", 3213, "201601", "PORT"],
  [2, "C", 2321, "201601", "DOCK"]],("id","type", "cost", "date", "ship"))

df_input.show()

+---+----+----+------+----+
| id|type|cost|  date|ship|
+---+----+----+------+----+
|  0|   A| 223|201603|PORT|
|  0|   A|  22|201602|PORT|
|  0|   A| 422|201601|DOCK|
|  1|   B|3213|201602|DOCK|
|  1|   B|3213|201601|PORT|
|  2|   C|2321|201601|DOCK|
+---+----+----+------+----+



In [20]:
# Result I
df_output = (df_input
             .groupby("id","type")
             .pivot("date")
             .agg(avg("cost"))
             .orderBy("id"))
df_output.show()

+---+----+------+------+------+
| id|type|201601|201602|201603|
+---+----+------+------+------+
|  0|   A| 422.0|  22.0| 223.0|
|  1|   B|3213.0|3213.0|  null|
|  2|   C|2321.0|  null|  null|
+---+----+------+------+------+



In [21]:
# Result II
df_output = (df_input
             .groupby("id","type")
             .pivot("date")
             .agg(collect_set("ship"))
             .orderBy("id"))
df_output.show()

+---+----+------+------+------+
| id|type|201601|201602|201603|
+---+----+------+------+------+
|  0|   A|[DOCK]|[PORT]|[PORT]|
|  1|   B|[PORT]|[DOCK]|    []|
|  2|   C|[DOCK]|    []|    []|
+---+----+------+------+------+



## 23. Pivoting on Multiple Columns <a id='23'></a>

In [24]:
df_input = spark.createDataFrame([
  [100,1,23,10],
  [100,2,45,11],
  [100,3,67,12],
  [100,4,78,13],
  [101,1,23,10],
  [101,2,45,13],
  [101,3,67,14],
  [101,4,78,15],
  [102,1,23,10],
  [102,2,45,11],
  [102,3,67,16],
  [102,4,78,18]],("id", "day", "price", "units"))

df_input.show()

+---+---+-----+-----+
| id|day|price|units|
+---+---+-----+-----+
|100|  1|   23|   10|
|100|  2|   45|   11|
|100|  3|   67|   12|
|100|  4|   78|   13|
|101|  1|   23|   10|
|101|  2|   45|   13|
|101|  3|   67|   14|
|101|  4|   78|   15|
|102|  1|   23|   10|
|102|  2|   45|   11|
|102|  3|   67|   16|
|102|  4|   78|   18|
+---+---+-----+-----+



In [26]:
# https://stackoverflow.com/questions/45035940/how-to-pivot-on-multiple-columns-in-spark-sql
df_output = (df_input
             .groupBy('id')
             .pivot('day')
             .agg(first('price').alias('price'),first('units').alias('unit')))
df_output.show()

+---+-------+------+-------+------+-------+------+-------+------+
| id|1_price|1_unit|2_price|2_unit|3_price|3_unit|4_price|4_unit|
+---+-------+------+-------+------+-------+------+-------+------+
|100|     23|    10|     45|    11|     67|    12|     78|    13|
|101|     23|    10|     45|    13|     67|    14|     78|    15|
|102|     23|    10|     45|    11|     67|    16|     78|    18|
+---+-------+------+-------+------+-------+------+-------+------+



## 24. Generating Exam Assessment Report <a id='24'></a>

In [27]:
df_input = spark.createDataFrame([
    [1,"Question1Text","Yes","abcde1",0,"(x1,y1)"],
    [2,"Question2Text","No","abcde1",0,"(x1,y1)"],
    [3,"Question3Text","3","abcde1",0,"(x1,y1)"],
    [1,"Question1Text","No","abcde2",0,"(x2,y2)"],
    [2,"Question2Text","Yes","abcde2",0,"(x2,y2)"]],("Qid","Question","AnswerText","ParticipantID","Assessment","GeoTag"))

df_input.show()

+---+-------------+----------+-------------+----------+-------+
|Qid|     Question|AnswerText|ParticipantID|Assessment| GeoTag|
+---+-------------+----------+-------------+----------+-------+
|  1|Question1Text|       Yes|       abcde1|         0|(x1,y1)|
|  2|Question2Text|        No|       abcde1|         0|(x1,y1)|
|  3|Question3Text|         3|       abcde1|         0|(x1,y1)|
|  1|Question1Text|        No|       abcde2|         0|(x2,y2)|
|  2|Question2Text|       Yes|       abcde2|         0|(x2,y2)|
+---+-------------+----------+-------------+----------+-------+



In [32]:
# https://stackoverflow.com/questions/47720822/how-to-pivot-on-arbitrary-column
df_output = df_input.withColumn("header", concat(lit("Qid_"), "Qid"))
df_output = (df_output
             .groupBy('ParticipantID', 'Assessment', 'GeoTag')
             .pivot('header')
             .agg(first('AnswerText')))
df_output.show()

+-------------+----------+-------+-----+-----+-----+
|ParticipantID|Assessment| GeoTag|Qid_1|Qid_2|Qid_3|
+-------------+----------+-------+-----+-----+-----+
|       abcde2|         0|(x2,y2)|   No|  Yes| null|
|       abcde1|         0|(x1,y1)|  Yes|   No|    3|
+-------------+----------+-------+-----+-----+-----+



## 25. Flattening Dataset from Long to Wide Format <a id='25'></a>

In [34]:
df_input = spark.createDataFrame([
["k1","v4","v7","d1"],
["k1","v5","v8","d2"],
["k1","v6","v9","d3"],
["k2","v12","v22","d1"],
["k2","v32","v42","d2"],
["k2","v11","v21","d3"]],("key","val1","val2","date"))

df_input.show()

+---+----+----+----+
|key|val1|val2|date|
+---+----+----+----+
| k1|  v4|  v7|  d1|
| k1|  v5|  v8|  d2|
| k1|  v6|  v9|  d3|
| k2| v12| v22|  d1|
| k2| v32| v42|  d2|
| k2| v11| v21|  d3|
+---+----+----+----+



In [35]:
# https://stackoverflow.com/questions/43349932/how-to-flatten-long-dataset-to-wide-format-pivot-with-no-join
df_output = (df_input
             .groupby("key")
             .pivot('date')
             .agg(first('val1').alias('v1'),first('val2').alias('v2')))
df_output.show()

+---+-----+-----+-----+-----+-----+-----+
|key|d1_v1|d1_v2|d2_v1|d2_v2|d3_v1|d3_v2|
+---+-----+-----+-----+-----+-----+-----+
| k2|  v12|  v22|  v32|  v42|  v11|  v21|
| k1|   v4|   v7|   v5|   v8|   v6|   v9|
+---+-----+-----+-----+-----+-----+-----+



## 26. Finding 1st and 2nd Bestsellers Per Genre <a id='26'></a>

In [36]:
df_input = spark.createDataFrame([
    [1,"Hunter Fields","romance",15],
    [2,"Leonard Lewis","thriller",81],
    [3,"Jason Dawson","thriller",90],
    [4,"Andre Grant","thriller",25],
    [5,"Earl Walton","romance",40],
    [6,"Alan Hanson","romance",24],
    [7,"Clyde Matthews","thriller",31],
    [8,"Josephine Leonard","thriller",1],
    [9,"Owen Boone","sci-fi",27],
    [10,"Max McBride","romance",75]], ("id","title","genre","quantity"))

df_input.show()

+---+-----------------+--------+--------+
| id|            title|   genre|quantity|
+---+-----------------+--------+--------+
|  1|    Hunter Fields| romance|      15|
|  2|    Leonard Lewis|thriller|      81|
|  3|     Jason Dawson|thriller|      90|
|  4|      Andre Grant|thriller|      25|
|  5|      Earl Walton| romance|      40|
|  6|      Alan Hanson| romance|      24|
|  7|   Clyde Matthews|thriller|      31|
|  8|Josephine Leonard|thriller|       1|
|  9|       Owen Boone|  sci-fi|      27|
| 10|      Max McBride| romance|      75|
+---+-----------------+--------+--------+



In [38]:
df_output = df_input.withColumn("rank", rank().over(Window.partitionBy("genre").orderBy(desc("quantity"))))
df_output = df_output.filter("rank <= 2")
df_output.show()                 

+---+-------------+--------+--------+----+
| id|        title|   genre|quantity|rank|
+---+-------------+--------+--------+----+
| 10|  Max McBride| romance|      75|   1|
|  5|  Earl Walton| romance|      40|   2|
|  3| Jason Dawson|thriller|      90|   1|
|  2|Leonard Lewis|thriller|      81|   2|
|  9|   Owen Boone|  sci-fi|      27|   1|
+---+-------------+--------+--------+----+



## 27. Calculating Gap Between Current And Highest Salaries Per Department <a id='27'></a>

In [39]:
df_input = spark.createDataFrame([
    [1,"Hunter Fields","IT",15],
    [2,"Leonard Lewis","Support",81],
    [3,"Jason Dawson","Support",90],
    [4,"Andre Grant","Support",25],
    [5,"Earl Walton","IT",40],
    [6,"Alan Hanson","IT",24],
    [7,"Clyde Matthews","Support",31],
    [8,"Josephine Leonard","Support",1],
    [9,"Owen Boone","HR",27],
    [10,"Max McBride","IT",75]],("id","name","department","salary"))

df_input.show()

+---+-----------------+----------+------+
| id|             name|department|salary|
+---+-----------------+----------+------+
|  1|    Hunter Fields|        IT|    15|
|  2|    Leonard Lewis|   Support|    81|
|  3|     Jason Dawson|   Support|    90|
|  4|      Andre Grant|   Support|    25|
|  5|      Earl Walton|        IT|    40|
|  6|      Alan Hanson|        IT|    24|
|  7|   Clyde Matthews|   Support|    31|
|  8|Josephine Leonard|   Support|     1|
|  9|       Owen Boone|        HR|    27|
| 10|      Max McBride|        IT|    75|
+---+-----------------+----------+------+



In [51]:
# Get max salary by department
df_max_salary = df_input.groupby("department").agg(max("salary"))
# Avoid department repetition when join
df_max_salary = df_max_salary.withColumnRenamed("department","max_department")
# Join
df_output = df_input.join(df_max_salary,df_input["department"] == df_max_salary["max_department"],"left")
# Drop repeated deparment
df_output = df_output.drop("max_department")
# Calculate difference
df_output = df_output.withColumn("diff",col("max(salary)")-col("salary")).drop("max(salary)")
df_output.show()

+---+-----------------+----------+------+----+
| id|             name|department|salary|diff|
+---+-----------------+----------+------+----+
|  9|       Owen Boone|        HR|    27|   0|
|  1|    Hunter Fields|        IT|    15|  60|
|  5|      Earl Walton|        IT|    40|  35|
|  6|      Alan Hanson|        IT|    24|  51|
| 10|      Max McBride|        IT|    75|   0|
|  2|    Leonard Lewis|   Support|    81|   9|
|  3|     Jason Dawson|   Support|    90|   0|
|  4|      Andre Grant|   Support|    25|  65|
|  7|   Clyde Matthews|   Support|    31|  59|
|  8|Josephine Leonard|   Support|     1|  89|
+---+-----------------+----------+------+----+



## 28. Calculating Running Total / Cumulative Sum <a id='28'></a>

In [52]:
df_input = spark.createDataFrame([
    [1,"IT",15],
    [2,"Support",81],
    [3,"Support",90],
    [4,"Support",25],
    [5,"IT",40],
    [6,"IT",24],
    [7,"Support",31],
    [8,"Support",1],
    [9,"HR",27],
    [10,"IT",75]],("time","department","items_sold"))

df_input.show()

+----+----------+----------+
|time|department|items_sold|
+----+----------+----------+
|   1|        IT|        15|
|   2|   Support|        81|
|   3|   Support|        90|
|   4|   Support|        25|
|   5|        IT|        40|
|   6|        IT|        24|
|   7|   Support|        31|
|   8|   Support|         1|
|   9|        HR|        27|
|  10|        IT|        75|
+----+----------+----------+



In [57]:
windowval = (Window.partitionBy('department').orderBy('time')
             .rangeBetween(Window.unboundedPreceding, 0))
df_output = df_input.withColumn('running_total', sum('items_sold').over(windowval))
df_output.show()

+----+----------+----------+-------------+
|time|department|items_sold|running_total|
+----+----------+----------+-------------+
|   9|        HR|        27|           27|
|   1|        IT|        15|           15|
|   5|        IT|        40|           55|
|   6|        IT|        24|           79|
|  10|        IT|        75|          154|
|   2|   Support|        81|           81|
|   3|   Support|        90|          171|
|   4|   Support|        25|          196|
|   7|   Support|        31|          227|
|   8|   Support|         1|          228|
+----+----------+----------+-------------+



## 29. Calculating Difference Between Consecutive Rows Per Window <a id='29'></a>

In [58]:
df_input = spark.createDataFrame([
    [1,"IT",15,15],
    [2,"Support",81,81],
    [3,"Support",90,171],
    [4,"Support",25,196],
    [5,"IT",40,55],
    [6,"IT",24,79],
    [7,"Support",31,227],
    [8,"Support",1,228],
    [9,"HR",27,27],
    [10,"IT",75,154]],("time","department","items_sold","running_total"))

df_input.show()

+----+----------+----------+-------------+
|time|department|items_sold|running_total|
+----+----------+----------+-------------+
|   1|        IT|        15|           15|
|   2|   Support|        81|           81|
|   3|   Support|        90|          171|
|   4|   Support|        25|          196|
|   5|        IT|        40|           55|
|   6|        IT|        24|           79|
|   7|   Support|        31|          227|
|   8|   Support|         1|          228|
|   9|        HR|        27|           27|
|  10|        IT|        75|          154|
+----+----------+----------+-------------+



In [59]:
# Estimate window by department and order by time
windowval = (Window.partitionBy('department').orderBy('time')
             .rangeBetween(Window.unboundedPreceding, 0))
# Apply cummulative sum
df_output = df_input.withColumn('running_total', sum('items_sold').over(windowval))
df_output.show()

+----+----------+----------+-------------+
|time|department|items_sold|running_total|
+----+----------+----------+-------------+
|   9|        HR|        27|           27|
|   1|        IT|        15|           15|
|   5|        IT|        40|           55|
|   6|        IT|        24|           79|
|  10|        IT|        75|          154|
|   2|   Support|        81|           81|
|   3|   Support|        90|          171|
|   4|   Support|        25|          196|
|   7|   Support|        31|          227|
|   8|   Support|         1|          228|
+----+----------+----------+-------------+



In [78]:
# Get lag value by department
df_output = df_output.withColumn("lag",lag(col("running_total")).over(Window.partitionBy("department").orderBy("time")))
# Fill null values of lag values without previous department
df_output = df_output.withColumn("lag",when(col("lag").isNull(),0).otherwise(col("lag")))
# Get difference
df_output = df_output.withColumn("diff",col("running_total")-col("lag")).drop("lag")
df_output.show()

+----+----------+----------+-------------+----+
|time|department|items_sold|running_total|diff|
+----+----------+----------+-------------+----+
|   9|        HR|        27|           27|  27|
|   1|        IT|        15|           15|  15|
|   5|        IT|        40|           55|  40|
|   6|        IT|        24|           79|  24|
|  10|        IT|        75|          154|  75|
|   2|   Support|        81|           81|  81|
|   3|   Support|        90|          171|  90|
|   4|   Support|        25|          196|  25|
|   7|   Support|        31|          227|  31|
|   8|   Support|         1|          228|   1|
+----+----------+----------+-------------+----+



## 30. Converting Arrays of Strings to String <a id='30'></a>

In [80]:
df_input = spark.createDataFrame([
    [1,["hello", "world"]]],("id","words"))

df_input.show()

+---+--------------+
| id|         words|
+---+--------------+
|  1|[hello, world]|
+---+--------------+



In [86]:
# Get Array elements
df_output = (df_input
             .withColumn("element_1",element_at("words",1))
             .withColumn("element_2",element_at("words",2)))
# Get solution
df_output = (df_output
             .withColumn("solution",concat_ws(" ",col("element_1"),col("element_2")))
             .drop("id","element_1","element_2"))
df_output.show()    

+--------------+-----------+
|         words|   solution|
+--------------+-----------+
|[hello, world]|hello world|
+--------------+-----------+

